In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Does nn.Conv2d init work well?

[Jump_to lesson 9 video](https://course19.fast.ai/videos/?lesson=9&t=21)

In [3]:
#export
from exp.nb_02 import *

def get_data():
    #path = datasets.download_data(MNIST_URL, ext='.gz')
    path = Path()/'mnist.pkl.gz'
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [4]:
torch.nn.modules.conv._ConvNd.reset_parameters??

Signature: torch.nn.modules.conv._ConvNd.reset_parameters(self) -> None
Docstring: <no docstring>
Source:   
    def reset_parameters(self) -> None:
        # Setting a=sqrt(5) in kaiming_uniform is the same as initializing with
        # uniform(-1/sqrt(k), 1/sqrt(k)), where k = weight.size(1) * prod(*kernel_size)
        # For more details see: https://github.com/pytorch/pytorch/issues/15314#issuecomment-477448573
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            if fan_in != 0:
                bound = 1 / math.sqrt(fan_in)
                init.uniform_(self.bias, -bound, bound)
File:      ~/mambaforge/lib/python3.9/site-packages/torch/nn/modules/conv.py
Type:      function


In [5]:
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)
x_train.shape,x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [7]:
n,*_ = x_train.shape
c = y_train.max()+1
nh = 32
n,c

(50000, tensor(10))

In [9]:
l1 = nn.Conv2d(1, nh, 5)

In [9]:
x = x_valid[:100]

In [10]:
x.shape

torch.Size([100, 1, 28, 28])

In [11]:
def stats(x): return x.mean(),x.std()

In [12]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [13]:
stats(l1.weight),stats(l1.bias)

((tensor(0.0052, grad_fn=<MeanBackward0>),
  tensor(0.1143, grad_fn=<StdBackward0>)),
 (tensor(-0.0108, grad_fn=<MeanBackward0>),
  tensor(0.1106, grad_fn=<StdBackward0>)))

**32 Filter out filter. 1 channel and 5x5 kernel**

In [14]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [15]:
l1.bias.shape

torch.Size([32])

In [17]:
t = l1(x)

In [18]:
stats(t)

(tensor(0.0019, grad_fn=<MeanBackward0>),
 tensor(0.6467, grad_fn=<StdBackward0>))

In [20]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(-0.0145, grad_fn=<MeanBackward0>),
 tensor(1.0269, grad_fn=<StdBackward0>))

In [21]:
import torch.nn.functional as F

In [22]:
def f1(x,a=0): return F.leaky_relu(l1(x),a)

**Variance is good**

In [23]:
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5615, grad_fn=<MeanBackward0>),
 tensor(1.1097, grad_fn=<StdBackward0>))

**Plain Pytorch results are not good at all. This is not good**

In [24]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2112, grad_fn=<MeanBackward0>),
 tensor(0.4290, grad_fn=<StdBackward0>))

In [25]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

**numel() brings the kernel size**

In [26]:
# receptive field size
rec_fs = l1.weight[0,0].numel()
rec_fs

25

**number of filters out, numbers of filters in; `nf` and `ni`

In [27]:
nf,ni,*_ = l1.weight.shape
nf,ni

(32, 1)

`fan_in` filters * number of receptive filter in , `fan_out` filters out * number of receptive filters out

In [28]:
fan_in  = ni*rec_fs
fan_out = nf*rec_fs
fan_in,fan_out

(25, 800)

check **stamp 12:34** for gain.

In [29]:
def gain(a): return math.sqrt(2.0 / (1 + a**2))

In [30]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

In [31]:
torch.zeros(10000).uniform_(-1,1).std()

tensor(0.5795)

In [32]:
1/math.sqrt(3.)

0.5773502691896258

In [33]:
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

In [34]:
kaiming2(l1.weight, a=0);
stats(f1(x))

(tensor(0.5510, grad_fn=<MeanBackward0>),
 tensor(1.1054, grad_fn=<StdBackward0>))

In [35]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.1938, grad_fn=<MeanBackward0>),
 tensor(0.4255, grad_fn=<StdBackward0>))

In [36]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

In [37]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [40]:
y = y_valid[:100].float()

**This is very good Jeremy runs the input thru the model and gets the stat. That was what i was looking for when experimenting on Panda3d**

In [42]:
t = m(x)
stats(t)

(tensor(0.0528, grad_fn=<MeanBackward0>),
 tensor(0.0089, grad_fn=<StdBackward0>))

**Results above is a verynot good, variance is almost zero**

In [43]:
l = mse(t,y)
l.backward()

In [44]:
stats(m[0].weight.grad)

(tensor(0.0052), tensor(0.0288))

In [45]:
init.kaiming_uniform_??

Signature:
init.kaiming_uniform_(
    tensor,
    a=0,
    mode='fan_in',
    nonlinearity='leaky_relu',
)
Source:   
def kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    uniform distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{U}(-\text{bound}, \text{bound})` where

    .. math::
        \text{bound} = \text{gain} \times \sqrt{\frac{3}{\text{fan\_mode}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (only
            used with ``'leaky_relu'``)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights

In [46]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [47]:
t = m(x)
stats(t)

(tensor(0.0601, grad_fn=<MeanBackward0>),
 tensor(0.2325, grad_fn=<StdBackward0>))

In [48]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(0.1327), tensor(0.2729))

## Export

In [49]:
!./notebook2script.py 02a_why_sqrt5.ipynb

Converted 02a_why_sqrt5.ipynb to exp/nb_02a.py
